In [5]:
!pip install pandas

In [1]:
import pandas as pd

file_path1 = "Marked Help Instances - 7x - anon - Marked Help Instances - 7x - anon.csv"
marked_df = pd.read_csv(file_path1)
file_path2 = "LiveLab_top_15_MATHia_7x_classes_anon.csv"
livelab_df = pd.read_csv(file_path2)

# Ensure timestamps are in datetime format
marked_df["helped_server_time2"] = pd.to_datetime(marked_df["helped_server_time2"], errors='coerce')
livelab_df["Time"] = pd.to_datetime(livelab_df["Time"], errors='coerce')

marked_df["helped_server_time2"] = marked_df["helped_server_time2"].dt.tz_localize(None)
livelab_df["Time"] = livelab_df["Time"].dt.tz_localize(None)

# Initialize helpedRightAfter column as False
livelab_df["helpedTransaction"] = False

# Process each student separately
for student_id in livelab_df["Anon Student Id"].unique():
    if student_id not in marked_df["anon_student_id"].values:
        continue  # Skip if no help data exists for the student
    
    # Get all LiveLab timestamps for this student (sorted)
    student_livelab_times = livelab_df[livelab_df["Anon Student Id"] == student_id].sort_values("Time")

    # Get all marked help timestamps for this student (sorted)
    student_help_times = sorted(marked_df[marked_df["anon_student_id"] == student_id]["helped_server_time2"])

    # Find the closest preceding timestamp for each help time
    last_marked_index = None  # Track which index was last marked
    for help_time in student_help_times:
        # Find all times before the help event
        before_help_times = student_livelab_times[student_livelab_times["Time"] < help_time]

        if not before_help_times.empty:
            # Get the latest action before the help time
            closest_time_index = before_help_times.index[-1]

            # Avoid marking multiple times for the same help event
            if last_marked_index != closest_time_index:
                livelab_df.at[closest_time_index, "helpedTransaction"] = True
                last_marked_index = closest_time_index  # Update last marked index
                
livelab_df.to_csv("Updated_LiveLab.csv", index=False)

# Display the updated DataFrame
print(livelab_df["helpedTransaction"])

true_students = livelab_df[livelab_df["helpedTransaction"] == True]
print(true_students[["Anon Student Id", "helpedTransaction"]])

0          False
1          False
2          False
3          False
4          False
           ...  
1650341    False
1650342    False
1650343    False
1650344    False
1650345    False
Name: helpedTransaction, Length: 1650346, dtype: bool
        Anon Student Id  helpedTransaction
2295               s247               True
5213               s221               True
8327               s334               True
10105              s144               True
14649              s132               True
...                 ...                ...
1645922             s59               True
1646374             s25               True
1648571            s160               True
1648714            s252               True
1649472            s341               True

[633 rows x 2 columns]


In [7]:
import pandas as pd

file_path1 = "Updated_LiveLab.csv"
livelab_df = pd.read_csv(file_path1)

livelab_df['requested_help'] = livelab_df['Help Level'].apply(lambda x: 1 if x > 0 else 0)
livelab_df['got_help'] = livelab_df['helpedTransaction'].apply(lambda x: 1 if x == True else 0)
correlation = livelab_df['requested_help'].corr(livelab_df['got_help'])
print(f"Correlation between requesting help and getting help: {correlation}")

Correlation between requesting help and getting help: -0.004414936609158632
